In [1]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from imutils import paths
import numpy as np
import os

Using TensorFlow backend.


In [2]:
IMG_HELP_PATH = '/Users/shankar/dev/code/ds/studies/data_science/dlcv'
import sys
sys.path.append(os.path.abspath(IMG_HELP_PATH))
from common.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from common.datasets import SimpleDatasetLoader
from common.nn.conv import FCHeadNet


In [3]:
DATASET = "../datasets/flowers17/images"
MODEL = "flowers17.model"

In [5]:
# Construct the image generator for the data augmentation
aug = ImageDataGenerator(
    rotation_range=30, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [6]:
# Grab the list of images that we'll be describing, then extract the
# class label names from the image paths
print("[INFO] Loading Images")
imagePaths = list(paths.list_images(DATASET))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

[INFO] Loading Images


In [7]:
# Initialize the image preprocessors
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

In [8]:
# Load the dataset from the disk then scale the raw pixel intensitites
# to the range [0, 1]
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [9]:
# Partition the data into training and testing splits using 75% of 
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [10]:
# Convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [11]:
# Load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

In [12]:
# Initialize the new head of the network, a set of FC layers followed
# by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

In [13]:
# Place the head FC model on top of the base model -- this will become
# the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

In [15]:
# Loop over all layers in the base model and freeze them so they will
# not be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [16]:
# Compile our model (this needs to be done after our setting our layers
# to being non-trainable)
print("[INFO] Compiling Model")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] Compiling Model


In [ ]:
# Train the head of the network for a few epochs (all other layers)
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY),
    epochs=25, steps_per_epoch=len(trainX)//32, verbose=1)

[INFO] training head
Epoch 1/25
31/31 [==============================] - 523s 17s/step - loss: 6.5616 - acc: 0.1900 - val_loss: 1.7633 - val_acc: 0.4000
Epoch 2/25
31/31 [==============================] - 521s 17s/step - loss: 2.3265 - acc: 0.3236 - val_loss: 1.6202 - val_acc: 0.4765
Epoch 3/25
31/31 [==============================] - 540s 17s/step - loss: 1.8373 - acc: 0.4284 - val_loss: 1.2419 - val_acc: 0.6029
Epoch 4/25
31/31 [==============================] - 528s 17s/step - loss: 1.6089 - acc: 0.5108 - val_loss: 0.8957 - val_acc: 0.7471
Epoch 5/25
31/31 [==============================] - 867s 28s/step - loss: 1.4074 - acc: 0.5478 - val_loss: 0.7453 - val_acc: 0.7971
Epoch 6/25
31/31 [==============================] - 508s 16s/step - loss: 1.3552 - acc: 0.5818 - val_loss: 0.7732 - val_acc: 0.7618
Epoch 7/25
31/31 [==============================] - 600s 19s/step - loss: 1.2068 - acc: 0.6087 - val_loss: 0.8156 - val_acc: 0.7353
Epoch 8/25
31/31 [==============================] - 518

In [ ]:
# Evaluate the network after initialization 
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), 
    target_names=classNames))

In [ ]:
# Save the model to disk
print("[INFO] Serializing Model")
model.save(MODEL)